In [1]:
import pandas as pd
import dask.dataframe as dd
import praw
from dask.diagnostics import ProgressBar
from tqdm import tqdm
import hashlib
from PIL import Image
import requests
from io import BytesIO
from shared_code.utility.schemas.pyarrow_schema import schema
from shared_code.utility.spark.set_environ import set_azure_env
import os
import torch
from datetime import datetime
import pandas as pd
from pmaw import PushshiftAPI
import time
set_azure_env()

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter
from shared_code.utility.scripts.blip_caption import BlipCaption

pbar = ProgressBar()
pbar.register()

tqdm.pandas()
tqdm.pandas(desc="global")

from tqdm.dask import TqdmCallback
cb = TqdmCallback(desc="global")
cb.register()

file_system = AzureFileStorageAdapter('data').get_file_storage()

In [2]:
%%time
 #["SFWRedheads","sfwpetite","SFWNextDoorGirls", "celebrities", "wallstreetbets", "fatsquirrelhate","CityPorn","EarthPorn","memes","spaceporn","trippinthroughtime","fatsquirrelhate","oldladiesbakingpies","itookapicture", "celebrities", "realasians", "KoreanHotties", "prettyasiangirls","AsianOfficeLady", "mildlypenis"]
subs = ["AsianOfficeLady"]

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time

extant_data = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
display(extant_data)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26270,twjucg,AsianOfficeLady,King_Oppai,#500,a woman in a suit and heels is sitting in a chair,1aae27f01d51c5e4aa70969e29fd9eed,/r/AsianOfficeLady/comments/twjucg/500/,https://i.redd.it/4lmcky1uamr81.jpg,1aae27f01d51c5e4aa70969e29fd9eed.jpg,D:\data\images\AsianOfficeLady\1aae27f01d51c5e...,D:\data\images\AsianOfficeLady\thumbnail\1aae2...,True,False
26271,tw2pnj,AsianOfficeLady,King_Oppai,#499,a woman in a short skirted skirt and high heel...,38a842e5241334b4ec98142120f51507,/r/AsianOfficeLady/comments/tw2pnj/499/,https://i.redd.it/axmwk1odqir81.jpg,38a842e5241334b4ec98142120f51507.jpg,D:\data\images\AsianOfficeLady\38a842e5241334b...,D:\data\images\AsianOfficeLady\thumbnail\38a84...,True,False
26272,tvqba6,AsianOfficeLady,King_Oppai,#498,woman in white shirt and black stockings with ...,45fd977542d10efb20c08a2a7fcf3062,/r/AsianOfficeLady/comments/tvqba6/498/,https://i.redd.it/hlxxvp9x5fr81.jpg,45fd977542d10efb20c08a2a7fcf3062.jpg,D:\data\images\AsianOfficeLady\45fd977542d10ef...,D:\data\images\AsianOfficeLady\thumbnail\45fd9...,True,False
26273,tv9wzv,AsianOfficeLady,King_Oppai,#497,asian woman in short skirt and black stockings...,7bd2a4eec57b53f820a0666181cd65e8,/r/AsianOfficeLady/comments/tv9wzv/497/,https://i.redd.it/rq4w4xwglbr81.jpg,7bd2a4eec57b53f820a0666181cd65e8.jpg,D:\data\images\AsianOfficeLady\7bd2a4eec57b53f...,D:\data\images\AsianOfficeLady\thumbnail\7bd2a...,True,False


CPU times: total: 750 ms
Wall time: 2.96 s


In [4]:
reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA',
								  user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

api: PushshiftAPI = PushshiftAPI(praw=reddit, num_workers=12)

Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


In [5]:
def filter_by_image_post(item) -> bool:
	return not (item['selftext'].__contains__('[removed]') or item['selftext'].__contains__('[deleted]') and not item[
		'url'].endswith('.jpg'))

In [6]:
def fetch_image(x: object) -> object:
	try:
		time.sleep(0.1)
		url = x['original_url']
		subreddit = x['subreddit']
		response = requests.get(url)
		md5 = hashlib.md5(response.content).hexdigest()
		os.makedirs(f"D:\\data\\images\\{subreddit}", exist_ok=True)
		out_path = f"D:\\data\\images\\{subreddit}\\{md5}.jpg"

		if os.path.exists(out_path):
			return out_path

		if md5 in list(extant_data['hash']):
			return out_path

		if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
			raw_image = Image.open(BytesIO(response.content))
			raw_image.save(out_path)
			raw_image.close()
			return out_path
		else:
			return ""
	except Exception as e:
		print(e)
		return ""

In [7]:
def make_thumbnail(x: object) -> str:
	image_path = x['path']
	md5 = x['hash']
	subreddit = x['subreddit']
	os.makedirs(f"D:\\data\\images\\{subreddit}\\thumbnail", exist_ok=True)
	out_path = f"D:\\data\\images\\{subreddit}\\thumbnail\\{md5}.jpg"
	if not os.path.exists(image_path):
		return ""
	if os.path.exists(out_path):
		return out_path
	img = Image.open(image_path)
	try:
		copied_image = img.copy()
		result_copy = copied_image.resize((512, 512))
		result_copy.save(out_path)
		result_copy.close()
		img.close()
		return out_path
	except Exception as e:
		print(e)
		return ""
	finally:
		img.close()

In [8]:
def exists(x: object) -> bool:
	try:
		image_path = os.path.exists(x['path'])
		thumbnail_path = os.path.exists(x['thumbnail_path'])
		return image_path and thumbnail_path
	except Exception as e:
		print(e)
		return False

In [9]:
def split_hash(x: object) -> str:
	try:
		name = x['image_name']
		return name.split('.')[0]
	except Exception as e:
		print(e)
		return ""

In [10]:
def caption_image(x: object, blip_captioning: BlipCaption) -> str:
	try:
		path = x['path']
		exists_image = bool(x['exists'])
		resulting_caption = x['caption']

		if not os.path.exists(path):
			return ""

		if not exists_image:
			return ""

		if len(resulting_caption) > 5:
			return resulting_caption

		resulting_caption = blip_captioning.caption_image(path)
		return resulting_caption

	except Exception as e:
		print(e)
		return ""
	finally:
		pass

In [11]:
def get_image_name(x: object) -> str:
	try:
		path = x['path']
		if path == "":
			return ""
		return os.path.basename(path)
	except Exception as e:
		print(e)
		return ""

In [12]:
%%time

all_posts = []
start_date: int = int(datetime.strptime('2021-01-01', '%Y-%m-%d').timestamp())
end_date: int = int(datetime.today().timestamp())
search_window=365,
for item in subs:
	posts = api.search_submissions(until=end_date, after=start_date, subreddit=item, limit=1000, filter_fn=filter_by_image_post, mem_safe=True, cache_dir='D:\\cache\\')
	[ all_posts.append(post) for post in posts ]

CPU times: total: 2.33 s
Wall time: 25.6 s


In [13]:
%%time

df = pd.DataFrame([item for item in all_posts])
display(df)

,comment_limit,comment_sort,_reddit,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,...,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,_fetched,_comments_by_id
0,2048,confidence,<praw.reddit.Reddit object at 0x000001E0D94499D0>,None,AsianOfficeLady,,t2_khb7s,False,None,0,...,None,False,https://i.redd.it/ejlwu8bjurw81.jpg,3924,1.651370e+09,0,None,False,False,{}
1,2048,confidence,<praw.reddit.Reddit object at 0x000001E0D94499D0>,None,AsianOfficeLady,,t2_khb7s,False,None,0,...,None,False,https://i.redd.it/nieclgv2aow81.jpg,3924,1.651327e+09,0,None,False,False,{}
2,2048,confidence,<praw.reddit.Reddit object at 0x000001E0D94499D0>,None,AsianOfficeLady,,t2_khb7s,False,None,0,...,None,False,https://i.redd.it/p08gw7gmpkw81.jpg,3924,1.651284e+09,0,None,False,False,{}
3,2048,confidence,<praw.reddit.Reddit object at 0x000001E0D94499D0>,None,AsianOfficeLady,,t2_khb7s,False,None,0,...,None,False,https://i.redd.it/g5epw6465hw81.jpg,3924,1.651241e+09,0,None,False,False,{}
4,2048,confidence,<praw.reddit.Reddit object at 0x000001E0D94499D0>,None,AsianOfficeLady,,t2_khb7s,False,None,0,...,None,False,https://i.redd.it/jetdbmqpkdw81.jpg,3924,1.651198e+09,0,None,False,False,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2048,confidence,<praw.reddit.Reddit object at 0x000001E0D94499D0>,None,AsianOfficeLady,,t2_khb7s,False,None,0,...,None,False,https://i.redd.it/b3vqo72z435a1.jpg,3924,1.670684e+09,0,None,False,False,{}
996,2048,confidence,<praw.reddit.Reddit object at 0x000001E0D94499D0>,None,AsianOfficeLady,,t2_khb7s,False,None,0,...,None,False,https://i.redd.it/d31cyrnikz4a1.jpg,3924,1.670641e+09,0,None,False,False,{}
997,2048,confidence,<praw.reddit.Reddit object at 0x000001E0D94499D0>,None,AsianOfficeLady,,t2_khb7s,False,None,0,...,None,False,https://i.redd.it/cfyz5na20w4a1.jpg,3924,1.670598e+09,0,None,False,False,{}
998,2048,confidence,<praw.reddit.Reddit object at 0x000001E0D94499D0>,None,AsianOfficeLady,,t2_khb7s,False,None,0,...,None,False,https://i.redd.it/3itnt8tlfs4a1.jpg,3924,1.670555e+09,0,None,False,False,{}


CPU times: total: 188 ms
Wall time: 219 ms


In [14]:
%%time

initial = pd.DataFrame({}, columns=schema.names)
initial['id'] = df['id']
initial['subreddit'] = df['subreddit']
initial['author'] = df['author']
initial['title'] = df['title']
initial['caption'] = ""
initial['hash'] = ""
initial['permalink'] = df['permalink']
initial['original_url'] = df['url']
initial['image_name'] = ""
initial['path'] = ""
initial['thumbnail_path'] = ""
initial['exists'] = False
initial['curated'] = False

filtered_initial = initial.where(initial['original_url'].str.endswith('.jpg')).dropna(how='all').reset_index().drop('index', axis=1)

display(filtered_initial)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,ufpk6z,AsianOfficeLady,King_Oppai,#550,,,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,,,,False,False
1,ufbo95,AsianOfficeLady,King_Oppai,#549,,,/r/AsianOfficeLady/comments/ufbo95/549/,https://i.redd.it/nieclgv2aow81.jpg,,,,False,False
2,uf12qi,AsianOfficeLady,King_Oppai,#548,,,/r/AsianOfficeLady/comments/uf12qi/548/,https://i.redd.it/p08gw7gmpkw81.jpg,,,,False,False
3,uemk5o,AsianOfficeLady,King_Oppai,#547,,,/r/AsianOfficeLady/comments/uemk5o/547/,https://i.redd.it/g5epw6465hw81.jpg,,,,False,False
4,ueb57q,AsianOfficeLady,King_Oppai,#546,,,/r/AsianOfficeLady/comments/ueb57q/546/,https://i.redd.it/jetdbmqpkdw81.jpg,,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,zhulkb,AsianOfficeLady,King_Oppai,#985,,,/r/AsianOfficeLady/comments/zhulkb/985/,https://i.redd.it/b3vqo72z435a1.jpg,,,,False,False
987,zhh1qb,AsianOfficeLady,King_Oppai,#984,,,/r/AsianOfficeLady/comments/zhh1qb/984/,https://i.redd.it/d31cyrnikz4a1.jpg,,,,False,False
988,zgzhzl,AsianOfficeLady,King_Oppai,#983,,,/r/AsianOfficeLady/comments/zgzhzl/983/,https://i.redd.it/cfyz5na20w4a1.jpg,,,,False,False
989,zgkxf8,AsianOfficeLady,King_Oppai,#982,,,/r/AsianOfficeLady/comments/zgkxf8/982/,https://i.redd.it/3itnt8tlfs4a1.jpg,,,,False,False


CPU times: total: 78.1 ms
Wall time: 93 ms


In [15]:
%%time

new_entries = filtered_initial.where(~filtered_initial['id'].isin(extant_data['id'])).dropna(how='all').reset_index().drop('index', axis=1)

display(new_entries)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,ufpk6z,AsianOfficeLady,King_Oppai,#550,,,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,,,,False,False
1,ufbo95,AsianOfficeLady,King_Oppai,#549,,,/r/AsianOfficeLady/comments/ufbo95/549/,https://i.redd.it/nieclgv2aow81.jpg,,,,False,False
2,ussmi2,AsianOfficeLady,King_Oppai,#585,,,/r/AsianOfficeLady/comments/ussmi2/585/,https://i.redd.it/hm5sv01yac091.jpg,,,,False,False
3,us26ex,AsianOfficeLady,King_Oppai,#584,,,/r/AsianOfficeLady/comments/us26ex/584/,https://i.redd.it/w35xt16165091.jpg,,,,False,False
4,urn3u5,AsianOfficeLady,King_Oppai,#583,,,/r/AsianOfficeLady/comments/urn3u5/583/,https://i.redd.it/wrrqtqtkl1091.jpg,,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,ugqaap,AsianOfficeLady,King_Oppai,#553,,,/r/AsianOfficeLady/comments/ugqaap/553/,https://i.redd.it/tju3k4kwj2x81.jpg,,,,False,False
257,ugf0ll,AsianOfficeLady,King_Oppai,#552,,,/r/AsianOfficeLady/comments/ugf0ll/552/,https://i.redd.it/5zf9nx5gzyw81.jpg,,,,False,False
258,ug0hqo,AsianOfficeLady,King_Oppai,#551,,,/r/AsianOfficeLady/comments/ug0hqo/551/,https://i.redd.it/wkrizvrzevw81.jpg,,,,False,False
259,ufpk6z,AsianOfficeLady,King_Oppai,#550,,,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,,,,False,False


CPU times: total: 78.1 ms
Wall time: 103 ms


In [16]:
%%time

new_entries_1 = new_entries.copy()

tqdm.pandas(desc="Fetch-Image")
with ProgressBar():
	new_entries_1['path'] = new_entries_1.progress_apply(lambda x: fetch_image(x), axis=1)

tqdm.pandas(desc="Get-Image-Name")
with ProgressBar():
	new_entries_1['image_name'] = new_entries_1.progress_apply(lambda x: get_image_name(x), axis=1)

tqdm.pandas(desc="Split-Hash")
with ProgressBar():
	new_entries_1['hash'] = new_entries_1.progress_apply(lambda x: split_hash(x), axis=1)

display(new_entries_1)

Fetch-Image:   5%|▍         | 13/261 [00:04<01:55,  2.16it/s]

image file is truncated (1 bytes not processed)


Fetch-Image:  19%|█▉        | 50/261 [00:18<00:59,  3.54it/s]

image file is truncated (1 bytes not processed)


Fetch-Image:  33%|███▎      | 87/261 [00:27<00:46,  3.74it/s]

...

Fetch-Image:  90%|█████████ | 235/261 [01:06<00:07,  3.69it/s]

image file is truncated (1 bytes not processed)


Split-Hash: 100%|██████████| 261/261 [00:00<00:00, 21729.56it/s]


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,ufpk6z,AsianOfficeLady,King_Oppai,#550,,9cf6b89609ec7c9c26bcb06a41e1bb1f,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,9cf6b89609ec7c9c26bcb06a41e1bb1f.jpg,D:\data\images\AsianOfficeLady\9cf6b89609ec7c9...,,False,False
1,ufbo95,AsianOfficeLady,King_Oppai,#549,,5e4ad6bc3b6af21a811cc0c91fdac890,/r/AsianOfficeLady/comments/ufbo95/549/,https://i.redd.it/nieclgv2aow81.jpg,5e4ad6bc3b6af21a811cc0c91fdac890.jpg,D:\data\images\AsianOfficeLady\5e4ad6bc3b6af21...,,False,False
2,ussmi2,AsianOfficeLady,King_Oppai,#585,,874fe479be1a696968be6ea0822e67be,/r/AsianOfficeLady/comments/ussmi2/585/,https://i.redd.it/hm5sv01yac091.jpg,874fe479be1a696968be6ea0822e67be.jpg,D:\data\images\AsianOfficeLady\874fe479be1a696...,,False,False
3,us26ex,AsianOfficeLady,King_Oppai,#584,,b32bd930779af7964e7537c33b9559e8,/r/AsianOfficeLady/comments/us26ex/584/,https://i.redd.it/w35xt16165091.jpg,b32bd930779af7964e7537c33b9559e8.jpg,D:\data\images\AsianOfficeLady\b32bd930779af79...,,False,False
4,urn3u5,AsianOfficeLady,King_Oppai,#583,,0fc684a023538e8eb97b4f64338b9af6,/r/AsianOfficeLady/comments/urn3u5/583/,https://i.redd.it/wrrqtqtkl1091.jpg,0fc684a023538e8eb97b4f64338b9af6.jpg,D:\data\images\AsianOfficeLady\0fc684a023538e8...,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,ugqaap,AsianOfficeLady,King_Oppai,#553,,28f6a8ca470803c3e4e18ba7b02b1ed0,/r/AsianOfficeLady/comments/ugqaap/553/,https://i.redd.it/tju3k4kwj2x81.jpg,28f6a8ca470803c3e4e18ba7b02b1ed0.jpg,D:\data\images\AsianOfficeLady\28f6a8ca470803c...,,False,False
257,ugf0ll,AsianOfficeLady,King_Oppai,#552,,47961ab8ac4e292562501a06e9f84d24,/r/AsianOfficeLady/comments/ugf0ll/552/,https://i.redd.it/5zf9nx5gzyw81.jpg,47961ab8ac4e292562501a06e9f84d24.jpg,D:\data\images\AsianOfficeLady\47961ab8ac4e292...,,False,False
258,ug0hqo,AsianOfficeLady,King_Oppai,#551,,57f69a9a8360af0706ed0566c5981665,/r/AsianOfficeLady/comments/ug0hqo/551/,https://i.redd.it/wkrizvrzevw81.jpg,57f69a9a8360af0706ed0566c5981665.jpg,D:\data\images\AsianOfficeLady\57f69a9a8360af0...,,False,False
259,ufpk6z,AsianOfficeLady,King_Oppai,#550,,9cf6b89609ec7c9c26bcb06a41e1bb1f,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,9cf6b89609ec7c9c26bcb06a41e1bb1f.jpg,D:\data\images\AsianOfficeLady\9cf6b89609ec7c9...,,False,False


CPU times: total: 18.8 s
Wall time: 1min 13s


In [17]:
%%time

new_entries_2 = new_entries_1.copy()

tqdm.pandas(desc="Create-Thumbnail")
with ProgressBar():
	new_entries_2['thumbnail_path'] = new_entries_2.progress_apply(lambda x: make_thumbnail(x), axis=1)

tqdm.pandas(desc="Checking-Exists")
with ProgressBar():
	new_entries_2['exists'] = new_entries_2.progress_apply(lambda x: exists(x), axis=1)

display(new_entries_2)

Checking-Exists: 100%|██████████| 261/261 [00:00<00:00, 3675.89it/s]


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,ufpk6z,AsianOfficeLady,King_Oppai,#550,,9cf6b89609ec7c9c26bcb06a41e1bb1f,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,9cf6b89609ec7c9c26bcb06a41e1bb1f.jpg,D:\data\images\AsianOfficeLady\9cf6b89609ec7c9...,D:\data\images\AsianOfficeLady\thumbnail\9cf6b...,True,False
1,ufbo95,AsianOfficeLady,King_Oppai,#549,,5e4ad6bc3b6af21a811cc0c91fdac890,/r/AsianOfficeLady/comments/ufbo95/549/,https://i.redd.it/nieclgv2aow81.jpg,5e4ad6bc3b6af21a811cc0c91fdac890.jpg,D:\data\images\AsianOfficeLady\5e4ad6bc3b6af21...,D:\data\images\AsianOfficeLady\thumbnail\5e4ad...,True,False
2,ussmi2,AsianOfficeLady,King_Oppai,#585,,874fe479be1a696968be6ea0822e67be,/r/AsianOfficeLady/comments/ussmi2/585/,https://i.redd.it/hm5sv01yac091.jpg,874fe479be1a696968be6ea0822e67be.jpg,D:\data\images\AsianOfficeLady\874fe479be1a696...,D:\data\images\AsianOfficeLady\thumbnail\874fe...,True,False
3,us26ex,AsianOfficeLady,King_Oppai,#584,,b32bd930779af7964e7537c33b9559e8,/r/AsianOfficeLady/comments/us26ex/584/,https://i.redd.it/w35xt16165091.jpg,b32bd930779af7964e7537c33b9559e8.jpg,D:\data\images\AsianOfficeLady\b32bd930779af79...,D:\data\images\AsianOfficeLady\thumbnail\b32bd...,True,False
4,urn3u5,AsianOfficeLady,King_Oppai,#583,,0fc684a023538e8eb97b4f64338b9af6,/r/AsianOfficeLady/comments/urn3u5/583/,https://i.redd.it/wrrqtqtkl1091.jpg,0fc684a023538e8eb97b4f64338b9af6.jpg,D:\data\images\AsianOfficeLady\0fc684a023538e8...,D:\data\images\AsianOfficeLady\thumbnail\0fc68...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,ugqaap,AsianOfficeLady,King_Oppai,#553,,28f6a8ca470803c3e4e18ba7b02b1ed0,/r/AsianOfficeLady/comments/ugqaap/553/,https://i.redd.it/tju3k4kwj2x81.jpg,28f6a8ca470803c3e4e18ba7b02b1ed0.jpg,D:\data\images\AsianOfficeLady\28f6a8ca470803c...,D:\data\images\AsianOfficeLady\thumbnail\28f6a...,True,False
257,ugf0ll,AsianOfficeLady,King_Oppai,#552,,47961ab8ac4e292562501a06e9f84d24,/r/AsianOfficeLady/comments/ugf0ll/552/,https://i.redd.it/5zf9nx5gzyw81.jpg,47961ab8ac4e292562501a06e9f84d24.jpg,D:\data\images\AsianOfficeLady\47961ab8ac4e292...,D:\data\images\AsianOfficeLady\thumbnail\47961...,True,False
258,ug0hqo,AsianOfficeLady,King_Oppai,#551,,57f69a9a8360af0706ed0566c5981665,/r/AsianOfficeLady/comments/ug0hqo/551/,https://i.redd.it/wkrizvrzevw81.jpg,57f69a9a8360af0706ed0566c5981665.jpg,D:\data\images\AsianOfficeLady\57f69a9a8360af0...,D:\data\images\AsianOfficeLady\thumbnail\57f69...,True,False
259,ufpk6z,AsianOfficeLady,King_Oppai,#550,,9cf6b89609ec7c9c26bcb06a41e1bb1f,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,9cf6b89609ec7c9c26bcb06a41e1bb1f.jpg,D:\data\images\AsianOfficeLady\9cf6b89609ec7c9...,D:\data\images\AsianOfficeLady\thumbnail\9cf6b...,True,False


CPU times: total: 2.97 s
Wall time: 3.11 s


In [18]:
%%time

blip = BlipCaption(1)

CPU times: total: 1min 4s
Wall time: 1min 9s


In [19]:
%%time

new_entries_final = new_entries_2.copy()
torch.cuda.empty_cache()

with pbar:
	ddf = dd.from_pandas(new_entries_final, npartitions=6)
	new_entries_final['caption'] = ddf.apply(lambda x: caption_image(x, blip), meta=('str', object), axis=1).compute()

torch.cuda.empty_cache()

display(new_entries_final)

[                                        ] | 0% Completed | 1.17 ms

global:   0%|          | 0/6 [00:00<?, ?it/s]

[                                        ] | 0% Completed | 23.33 sms

D:\workspaces\General\venv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[######                                  ] | 16% Completed | 302.20 s

...

[##########################              ] | 66% Completed | 312.91 s

D:\workspaces\General\venv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[########################################] | 100% Completed | 314.99 s


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,ufpk6z,AsianOfficeLady,King_Oppai,#550,a woman in a white shirt and black skirt sitti...,9cf6b89609ec7c9c26bcb06a41e1bb1f,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,9cf6b89609ec7c9c26bcb06a41e1bb1f.jpg,D:\data\images\AsianOfficeLady\9cf6b89609ec7c9...,D:\data\images\AsianOfficeLady\thumbnail\9cf6b...,True,False
1,ufbo95,AsianOfficeLady,King_Oppai,#549,a woman in a white shirt and red panties is po...,5e4ad6bc3b6af21a811cc0c91fdac890,/r/AsianOfficeLady/comments/ufbo95/549/,https://i.redd.it/nieclgv2aow81.jpg,5e4ad6bc3b6af21a811cc0c91fdac890.jpg,D:\data\images\AsianOfficeLady\5e4ad6bc3b6af21...,D:\data\images\AsianOfficeLady\thumbnail\5e4ad...,True,False
2,ussmi2,AsianOfficeLady,King_Oppai,#585,a woman in a short skirt and tight stockings i...,874fe479be1a696968be6ea0822e67be,/r/AsianOfficeLady/comments/ussmi2/585/,https://i.redd.it/hm5sv01yac091.jpg,874fe479be1a696968be6ea0822e67be.jpg,D:\data\images\AsianOfficeLady\874fe479be1a696...,D:\data\images\AsianOfficeLady\thumbnail\874fe...,True,False
3,us26ex,AsianOfficeLady,King_Oppai,#584,a woman in a skirted skirt and stockings is sh...,b32bd930779af7964e7537c33b9559e8,/r/AsianOfficeLady/comments/us26ex/584/,https://i.redd.it/w35xt16165091.jpg,b32bd930779af7964e7537c33b9559e8.jpg,D:\data\images\AsianOfficeLady\b32bd930779af79...,D:\data\images\AsianOfficeLady\thumbnail\b32bd...,True,False
4,urn3u5,AsianOfficeLady,King_Oppai,#583,a woman in a white dress and red shoes is posi...,0fc684a023538e8eb97b4f64338b9af6,/r/AsianOfficeLady/comments/urn3u5/583/,https://i.redd.it/wrrqtqtkl1091.jpg,0fc684a023538e8eb97b4f64338b9af6.jpg,D:\data\images\AsianOfficeLady\0fc684a023538e8...,D:\data\images\AsianOfficeLady\thumbnail\0fc68...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,ugqaap,AsianOfficeLady,King_Oppai,#553,a woman in a short dress standing in front of ...,28f6a8ca470803c3e4e18ba7b02b1ed0,/r/AsianOfficeLady/comments/ugqaap/553/,https://i.redd.it/tju3k4kwj2x81.jpg,28f6a8ca470803c3e4e18ba7b02b1ed0.jpg,D:\data\images\AsianOfficeLady\28f6a8ca470803c...,D:\data\images\AsianOfficeLady\thumbnail\28f6a...,True,False
257,ugf0ll,AsianOfficeLady,King_Oppai,#552,a woman in a white dress and stockings is sitt...,47961ab8ac4e292562501a06e9f84d24,/r/AsianOfficeLady/comments/ugf0ll/552/,https://i.redd.it/5zf9nx5gzyw81.jpg,47961ab8ac4e292562501a06e9f84d24.jpg,D:\data\images\AsianOfficeLady\47961ab8ac4e292...,D:\data\images\AsianOfficeLady\thumbnail\47961...,True,False
258,ug0hqo,AsianOfficeLady,King_Oppai,#551,a woman in a red dress is standing in a room,57f69a9a8360af0706ed0566c5981665,/r/AsianOfficeLady/comments/ug0hqo/551/,https://i.redd.it/wkrizvrzevw81.jpg,57f69a9a8360af0706ed0566c5981665.jpg,D:\data\images\AsianOfficeLady\57f69a9a8360af0...,D:\data\images\AsianOfficeLady\thumbnail\57f69...,True,False
259,ufpk6z,AsianOfficeLady,King_Oppai,#550,a woman in a white shirt and black skirt sitti...,9cf6b89609ec7c9c26bcb06a41e1bb1f,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,9cf6b89609ec7c9c26bcb06a41e1bb1f.jpg,D:\data\images\AsianOfficeLady\9cf6b89609ec7c9...,D:\data\images\AsianOfficeLady\thumbnail\9cf6b...,True,False


CPU times: total: 11min 37s
Wall time: 6min 5s


In [20]:
%%time

tqdm.pandas(desc="Filtering")
writeable_entries = new_entries_final\
	.where(~new_entries_final['id'].isin(extant_data['id']))\
	.dropna(how='all')\
	.reset_index()\
	.drop('index', axis=1)

display(writeable_entries)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,ufpk6z,AsianOfficeLady,King_Oppai,#550,a woman in a white shirt and black skirt sitti...,9cf6b89609ec7c9c26bcb06a41e1bb1f,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,9cf6b89609ec7c9c26bcb06a41e1bb1f.jpg,D:\data\images\AsianOfficeLady\9cf6b89609ec7c9...,D:\data\images\AsianOfficeLady\thumbnail\9cf6b...,True,False
1,ufbo95,AsianOfficeLady,King_Oppai,#549,a woman in a white shirt and red panties is po...,5e4ad6bc3b6af21a811cc0c91fdac890,/r/AsianOfficeLady/comments/ufbo95/549/,https://i.redd.it/nieclgv2aow81.jpg,5e4ad6bc3b6af21a811cc0c91fdac890.jpg,D:\data\images\AsianOfficeLady\5e4ad6bc3b6af21...,D:\data\images\AsianOfficeLady\thumbnail\5e4ad...,True,False
2,ussmi2,AsianOfficeLady,King_Oppai,#585,a woman in a short skirt and tight stockings i...,874fe479be1a696968be6ea0822e67be,/r/AsianOfficeLady/comments/ussmi2/585/,https://i.redd.it/hm5sv01yac091.jpg,874fe479be1a696968be6ea0822e67be.jpg,D:\data\images\AsianOfficeLady\874fe479be1a696...,D:\data\images\AsianOfficeLady\thumbnail\874fe...,True,False
3,us26ex,AsianOfficeLady,King_Oppai,#584,a woman in a skirted skirt and stockings is sh...,b32bd930779af7964e7537c33b9559e8,/r/AsianOfficeLady/comments/us26ex/584/,https://i.redd.it/w35xt16165091.jpg,b32bd930779af7964e7537c33b9559e8.jpg,D:\data\images\AsianOfficeLady\b32bd930779af79...,D:\data\images\AsianOfficeLady\thumbnail\b32bd...,True,False
4,urn3u5,AsianOfficeLady,King_Oppai,#583,a woman in a white dress and red shoes is posi...,0fc684a023538e8eb97b4f64338b9af6,/r/AsianOfficeLady/comments/urn3u5/583/,https://i.redd.it/wrrqtqtkl1091.jpg,0fc684a023538e8eb97b4f64338b9af6.jpg,D:\data\images\AsianOfficeLady\0fc684a023538e8...,D:\data\images\AsianOfficeLady\thumbnail\0fc68...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,ugqaap,AsianOfficeLady,King_Oppai,#553,a woman in a short dress standing in front of ...,28f6a8ca470803c3e4e18ba7b02b1ed0,/r/AsianOfficeLady/comments/ugqaap/553/,https://i.redd.it/tju3k4kwj2x81.jpg,28f6a8ca470803c3e4e18ba7b02b1ed0.jpg,D:\data\images\AsianOfficeLady\28f6a8ca470803c...,D:\data\images\AsianOfficeLady\thumbnail\28f6a...,True,False
257,ugf0ll,AsianOfficeLady,King_Oppai,#552,a woman in a white dress and stockings is sitt...,47961ab8ac4e292562501a06e9f84d24,/r/AsianOfficeLady/comments/ugf0ll/552/,https://i.redd.it/5zf9nx5gzyw81.jpg,47961ab8ac4e292562501a06e9f84d24.jpg,D:\data\images\AsianOfficeLady\47961ab8ac4e292...,D:\data\images\AsianOfficeLady\thumbnail\47961...,True,False
258,ug0hqo,AsianOfficeLady,King_Oppai,#551,a woman in a red dress is standing in a room,57f69a9a8360af0706ed0566c5981665,/r/AsianOfficeLady/comments/ug0hqo/551/,https://i.redd.it/wkrizvrzevw81.jpg,57f69a9a8360af0706ed0566c5981665.jpg,D:\data\images\AsianOfficeLady\57f69a9a8360af0...,D:\data\images\AsianOfficeLady\thumbnail\57f69...,True,False
259,ufpk6z,AsianOfficeLady,King_Oppai,#550,a woman in a white shirt and black skirt sitti...,9cf6b89609ec7c9c26bcb06a41e1bb1f,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,9cf6b89609ec7c9c26bcb06a41e1bb1f.jpg,D:\data\images\AsianOfficeLady\9cf6b89609ec7c9...,D:\data\images\AsianOfficeLady\thumbnail\9cf6b...,True,False


CPU times: total: 78.1 ms
Wall time: 66 ms


In [21]:
%%time

combined_result = pd.concat([extant_data, writeable_entries])

display(combined_result)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,ugqaap,AsianOfficeLady,King_Oppai,#553,a woman in a short dress standing in front of ...,28f6a8ca470803c3e4e18ba7b02b1ed0,/r/AsianOfficeLady/comments/ugqaap/553/,https://i.redd.it/tju3k4kwj2x81.jpg,28f6a8ca470803c3e4e18ba7b02b1ed0.jpg,D:\data\images\AsianOfficeLady\28f6a8ca470803c...,D:\data\images\AsianOfficeLady\thumbnail\28f6a...,True,False
257,ugf0ll,AsianOfficeLady,King_Oppai,#552,a woman in a white dress and stockings is sitt...,47961ab8ac4e292562501a06e9f84d24,/r/AsianOfficeLady/comments/ugf0ll/552/,https://i.redd.it/5zf9nx5gzyw81.jpg,47961ab8ac4e292562501a06e9f84d24.jpg,D:\data\images\AsianOfficeLady\47961ab8ac4e292...,D:\data\images\AsianOfficeLady\thumbnail\47961...,True,False
258,ug0hqo,AsianOfficeLady,King_Oppai,#551,a woman in a red dress is standing in a room,57f69a9a8360af0706ed0566c5981665,/r/AsianOfficeLady/comments/ug0hqo/551/,https://i.redd.it/wkrizvrzevw81.jpg,57f69a9a8360af0706ed0566c5981665.jpg,D:\data\images\AsianOfficeLady\57f69a9a8360af0...,D:\data\images\AsianOfficeLady\thumbnail\57f69...,True,False
259,ufpk6z,AsianOfficeLady,King_Oppai,#550,a woman in a white shirt and black skirt sitti...,9cf6b89609ec7c9c26bcb06a41e1bb1f,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,9cf6b89609ec7c9c26bcb06a41e1bb1f.jpg,D:\data\images\AsianOfficeLady\9cf6b89609ec7c9...,D:\data\images\AsianOfficeLady\thumbnail\9cf6b...,True,False


CPU times: total: 62.5 ms
Wall time: 60 ms


In [22]:
combined_result.reindex()
combined_result.to_csv('processed_raw_data.csv', index=False)
f = AzureFileStorageAdapter('data').get_file_storage()
f.put('processed_raw_data.csv','data/processed_raw_data.csv', overwrite=True)

[None]

In [23]:
final = pd.read_csv(f.open('data/processed_raw_data.csv'))
final.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

In [24]:
final = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)
display(final)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26531,ugqaap,AsianOfficeLady,King_Oppai,#553,a woman in a short dress standing in front of ...,28f6a8ca470803c3e4e18ba7b02b1ed0,/r/AsianOfficeLady/comments/ugqaap/553/,https://i.redd.it/tju3k4kwj2x81.jpg,28f6a8ca470803c3e4e18ba7b02b1ed0.jpg,D:\data\images\AsianOfficeLady\28f6a8ca470803c...,D:\data\images\AsianOfficeLady\thumbnail\28f6a...,True,False
26532,ugf0ll,AsianOfficeLady,King_Oppai,#552,a woman in a white dress and stockings is sitt...,47961ab8ac4e292562501a06e9f84d24,/r/AsianOfficeLady/comments/ugf0ll/552/,https://i.redd.it/5zf9nx5gzyw81.jpg,47961ab8ac4e292562501a06e9f84d24.jpg,D:\data\images\AsianOfficeLady\47961ab8ac4e292...,D:\data\images\AsianOfficeLady\thumbnail\47961...,True,False
26533,ug0hqo,AsianOfficeLady,King_Oppai,#551,a woman in a red dress is standing in a room,57f69a9a8360af0706ed0566c5981665,/r/AsianOfficeLady/comments/ug0hqo/551/,https://i.redd.it/wkrizvrzevw81.jpg,57f69a9a8360af0706ed0566c5981665.jpg,D:\data\images\AsianOfficeLady\57f69a9a8360af0...,D:\data\images\AsianOfficeLady\thumbnail\57f69...,True,False
26534,ufpk6z,AsianOfficeLady,King_Oppai,#550,a woman in a white shirt and black skirt sitti...,9cf6b89609ec7c9c26bcb06a41e1bb1f,/r/AsianOfficeLady/comments/ufpk6z/550/,https://i.redd.it/ejlwu8bjurw81.jpg,9cf6b89609ec7c9c26bcb06a41e1bb1f.jpg,D:\data\images\AsianOfficeLady\9cf6b89609ec7c9...,D:\data\images\AsianOfficeLady\thumbnail\9cf6b...,True,False
